In [175]:
import numpy as np 
import pandas as pd
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV

In [142]:
# Carregando os dados 
path = "./titanic/"
data_train = pd.read_csv(path + "train.csv")
data_test = pd.read_csv(path + "test.csv")

In [143]:
data_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [144]:
data_train.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [145]:
# separa a coluna de labels
labels = data_train["Survived"]

# retira a coluna labels e outras sem interesse (nome) do dados de treinamento
data_train.drop(['Survived','Name','Ticket'], axis = 1, inplace=True)

data_train['Sex'] = data_train['Sex'].map({'male':0,'female':1}, na_action=None)

data_train['Embarked'] = data_train['Embarked'].map({'C':0,'Q':1,'S':2}, na_action=None)

data_test.drop(['Name','Ticket'], axis = 1, inplace=True)
data_test['Sex'] = data_test['Sex'].map({'male':0,'female':1}, na_action=None)
data_test['Embarked'] = data_test['Embarked'].map({'C':0,'Q':1,'S':2}, na_action=None)

In [146]:
data_train.head()

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,1,3,0,22.0,1,0,7.2500,NaN,2.0
1,2,1,1,38.0,1,0,71.2833,C85,0.0
2,3,3,1,26.0,0,0,7.9250,NaN,2.0
3,4,1,1,35.0,1,0,53.1000,C123,2.0
4,5,3,0,35.0,0,0,8.0500,NaN,2.0


In [147]:
data_train['Cabin'].fillna('Z',inplace=True)
data_train['Cabin'] = data_train['Cabin'].str[:1]
data_train['Cabin'] = data_train['Cabin'].map({'A':0, 'B':1, 'C':2, 'D':3, 'E':4, 'F':5, 'G':6, 'T':7, 'Z':8}, na_action=None)

data_test['Cabin'].fillna('Z',inplace=True)
data_test['Cabin'] = data_test['Cabin'].str[:1]
data_test['Cabin'] = data_test['Cabin'].map({'A':0, 'B':1, 'C':2, 'D':3, 'E':4, 'F':5, 'G':6, 'T':7, 'Z':8}, na_action=None)

In [148]:
data_train.head()

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,1,3,0,22.0,1,0,7.2500,8,2.0
1,2,1,1,38.0,1,0,71.2833,2,0.0
2,3,3,1,26.0,0,0,7.9250,8,2.0
3,4,1,1,35.0,1,0,53.1000,2,2.0
4,5,3,0,35.0,0,0,8.0500,8,2.0


In [149]:
data_train.isnull().sum()

PassengerId      0
Pclass           0
Sex              0
Age            177
SibSp            0
Parch            0
Fare             0
Cabin            0
Embarked         2
dtype: int64

In [150]:
data_train['Age'].fillna(data_train['Age'].mean(),inplace=True)
data_train['Embarked'].fillna(data_train['Embarked'].mean(),inplace=True)

In [151]:
data_test.isnull().sum()

PassengerId     0
Pclass          0
Sex             0
Age            86
SibSp           0
Parch           0
Fare            1
Cabin           0
Embarked        0
dtype: int64

In [152]:
data_test['Age'].fillna(data_test['Age'].mean(),inplace=True)
data_test['Fare'].fillna(data_test['Fare'].mean(),inplace=True)

In [156]:
data_train.columns

Index(['PassengerId', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare',
       'Cabin', 'Embarked'],
      dtype='object')

In [158]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler() 
scaled_values = scaler.fit_transform(data_train[['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare',
       'Cabin', 'Embarked']]) 
data_train[['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare',
       'Cabin', 'Embarked']] = scaled_values

scaled_values = scaler.fit_transform(data_test[['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare',
       'Cabin', 'Embarked']]) 
data_test[['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare',
       'Cabin', 'Embarked']] = scaled_values

In [159]:
data_train

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,1,1.0,0.0,0.271174,0.125,0.000000,0.014151,1.000,1.0
1,2,0.0,1.0,0.472229,0.125,0.000000,0.139136,0.250,0.0
2,3,1.0,1.0,0.321438,0.000,0.000000,0.015469,1.000,1.0
3,4,0.0,1.0,0.434531,0.125,0.000000,0.103644,0.250,1.0
4,5,1.0,0.0,0.434531,0.000,0.000000,0.015713,1.000,1.0
...,...,...,...,...,...,...,...,...,...
886,887,0.5,0.0,0.334004,0.000,0.000000,0.025374,1.000,1.0
887,888,0.0,1.0,0.233476,0.000,0.000000,0.058556,0.125,1.0
888,889,1.0,1.0,0.367921,0.125,0.333333,0.045771,1.000,1.0
889,890,0.0,0.0,0.321438,0.000,0.000000,0.058556,0.250,0.0


In [179]:
data_fields = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare',  'Cabin', 'Embarked']

In [180]:
### MLPClassifier
mlp_model = MLPClassifier(solver='adam', # ‘lbfgs’, ‘sgd’, ‘adam’
                          alpha=1e-5,    # regularização L2 ridge
                          hidden_layer_sizes=(512, 64),  #default (100,) 
                          random_state=1, 
                          max_iter=100)

mlp_model.fit(data_train[data_fields], labels)                         

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPClassifier(alpha=1e-05, hidden_layer_sizes=(512, 64), max_iter=100,
              random_state=1)

In [181]:
pred_mlp =  mlp_model.predict(data_test[data_fields]) # obteve score de 0.97567

In [182]:
output = pd.DataFrame({'PassengerId': data_test.PassengerId, 'Survived': pred_mlp})
output.to_csv('my_submissionMLP.csv', index=False)

In [183]:
parameter_space = {
    'hidden_layer_sizes': [(512,64), (256,32), (100,)],
    'activation': ['tanh', 'relu'],
    'solver': ['sgd', 'adam'],
    'alpha': [0.0001, 0.05],
    'learning_rate': ['constant','adaptive'],
}
clf = GridSearchCV(mlp_model, parameter_space, n_jobs=-1, cv=3)
clf.fit(data_train[data_fields], labels)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(


GridSearchCV(cv=3,
             estimator=MLPClassifier(alpha=1e-05, hidden_layer_sizes=(512, 64),
                                     max_iter=100, random_state=1),
             n_jobs=-1,
             param_grid={'activation': ['tanh', 'relu'],
                         'alpha': [0.0001, 0.05],
                         'hidden_layer_sizes': [(512, 64), (256, 32), (100,)],
                         'learning_rate': ['constant', 'adaptive'],
                         'solver': ['sgd', 'adam']})

In [184]:
pred_mlp =  clf.predict(data_test[data_fields]) 
output = pd.DataFrame({'PassengerId': data_test.PassengerId, 'Survived': pred_mlp})
output.to_csv('my_submissionMLP.csv', index=False)